# Initial Imports

In [ ]:
!pip install -q tensorflow-recommenders

In [ ]:
import os
import pprint
import tempfile

from typing import Dict, Text

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import csv
import requests
from bs4 import BeautifulSoup

In [ ]:
import tensorflow_recommenders as tfrs

# Loading Data

In [ ]:
response = requests.get("https://cdn.discordapp.com/attachments/1106183330547892234/1110213188412243998/indonesia.html")
response_content = response.content
soup = BeautifulSoup(response_content, 'html.parser')

table = soup.find('table')

# headers = [header.text.strip() for header in table.find_all('th')]
headers = ['id', 'date', 'event_name', 'location', 'description', 'type', 'interested']

rows = []
id = 1
for tr in table.find_all('tr'):
    row = []

    # pass featured cards
    if (tr.find('svg', class_="svg-inline--fa fa-star fa-w-18 me-1 text-orange-l") != None):
      continue

    row.append(str(id))

    data = tr.find('div', class_='small fw-500')
    row.append(data.text) if data else row.append('')

    data = tr.find('span', class_='d-block')
    row.append(data.text) if data else row.append('')

    data = tr.find('div', class_='small fw-500 venue')
    row.append(data.text.replace('\xa0•\xa0Online', '')) if data else row.append('')

    data = tr.find('div', class_='small text-wrap text-break')
    row.append(data.text) if data else row.append('')

    # not enough data
    if (len(''.join(row[1:5]).strip()) == 0):
      continue

    data = tr.find('td', class_='col-12 small text-muted mb-2')
    row.append([tag.text.strip() for tag in data if tag.text.strip() != '']) if data else row.append('')

    data = tr.find('a', class_='small fw-500 text-decoration-none px-2 xn')
    row.append(data.text) if data else row.append('')

    rows.append(row)

    id += 1

# Specify the CSV file path
csv_file = 'events.csv'

# Write the data to the CSV file
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerows(rows)

event_count = len(rows)

In [59]:
key='9aadb790'
user_count=1000
generate_count=1000
event_count=400

ratings_df = pd.read_csv(f"https://my.api.mockaroo.com/rating.json?key={key}&count={generate_count}&user_id={user_count}&event_id={event_count}")

# # drop duplicated
duplicated_series = ratings_df.duplicated(subset=['user_id', 'event_id'], keep=False)
for row in range(0,len(duplicated_series)):
    if duplicated_series[row] == True:
        ratings_df.drop(index=row, inplace=True)

ratings_df.reset_index(drop=True, inplace=True)

# check for duplicated
duplicated_series = ratings_df.duplicated(subset=['user_id', 'event_id'], keep=False)
for row in range(0,len(duplicated_series)):
    if duplicated_series[row] == True:
        print(row)


ratings_df.to_csv('ratings.csv', sep=',', encoding='utf-8')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df))

events_df = pd.read_csv("events.csv")

non_numeric_cols = events_df.select_dtypes(exclude=['int64', 'float64', 'string']).columns
events_df[non_numeric_cols] = events_df[non_numeric_cols].astype(str)
events_df['interested'] = events_df['interested'].fillna(0).astype(int)

events = tf.data.Dataset.from_tensor_slices(dict(events_df))

## ID as str

In [ ]:
ratings_df = pd.read_csv(f"https://api.mockaroo.com/api/9ba52b20?count=1000&key=9aadb790&user_id=1000&event_id={event_count}")
events_df = pd.read_csv("/content/events.csv")
ratings_df[['event_id','user_id']] = ratings_df[['event_id','user_id']].astype(str)
events_df = events_df.astype(str)

ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_df))
events = tf.data.Dataset.from_tensor_slices(dict(events_df))

In [60]:
print(ratings_df.info(verbose=True))
print(events_df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      1000 non-null   int64
 1   user_rating  1000 non-null   int64
 2   event_id     1000 non-null   int64
dtypes: int64(3)
memory usage: 23.6 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           400 non-null    int64 
 1   date         400 non-null    object
 2   event_name   400 non-null    object
 3   location     400 non-null    object
 4   description  400 non-null    object
 5   type         400 non-null    object
 6   interested   400 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 22.0+ KB
None


In [61]:
ratings_df.head()

,user_id,user_rating,event_id
0,40,2,210
1,662,5,228
2,212,1,26
3,10,4,281
4,316,1,278


In [62]:
events_df.head()

,id,date,event_name,location,description,type,interested
0,1,"Wed, 24 - Fri, 26 May 2023",Tyre & Rubber Indonesia,Jakarta,The Tyre & Rubber Indonesia is the only dedica...,"['Tradeshow', 'Auto & Automotive', 'Rubber & T...",304
1,2,"Wed, 24 - Fri, 26 May 2023","Indonesia International Construction, Infrastr...",Jakarta,Con-Mine – GET your ultimate business chances ...,"['Tradeshow', 'Building & Construction']",102
2,3,"Sun, 11 - Wed, 14 Jun 2023",ICM Triennial Congress,Bali,It will be six years since our last in-person ...,"['Conference', 'Medical & Pharma', 'Investment']",35
3,4,"Wed, 07 - Thu, 08 Jun 2023",World AI Show - Jakarta (WAIS),Jakarta,Augmenting digital transformtaion with AI,"['Tradeshow', 'IT & Technology']",34
4,5,"Wed, 14 - Sat, 17 Jun 2023",Bali & Beyond Travel Fair (BBTF),Bali,Bali and Beyond Travel Fair (BBTF) is Indonesi...,"['Tradeshow', 'Travel & Tourism']",116


In [63]:
ratings = ratings.map(lambda x: {
    "event_id": x["event_id"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})
events = events.map(lambda x: x["id"])
print(list(ratings.take(1).as_numpy_iterator()))
print(list(events.take(1).as_numpy_iterator()))

[{'event_id': 210, 'user_id': 40, 'user_rating': 2}]
[1]


In [64]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
n = tf.data.experimental.cardinality(shuffled)

train = shuffled.take(int(n*4/5))
test = shuffled.skip(int(n*4/5)).take(int(n*1/5))

In [65]:
event_names = events.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_event_ids = np.unique(np.concatenate(list(event_names)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print(unique_event_ids[:10])
print(unique_user_ids[:10])

[ 1  2  3  4  5  6  7  8  9 10]
[ 2  3  5  6  7  8 10 11 12 13]


# Creating Models

## Retrieval Model

In [66]:
embedding_dimension = 32
# Candidate tower
event_model = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(
        vocabulary=unique_event_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_event_ids) + 1, embedding_dimension)
])

# Query tower
user_model = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(
        vocabulary=unique_user_ids, mask_token=None),
        # We add an additional embedding to account for unknown tokens.
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

task = tfrs.tasks.Retrieval(
    metrics = tfrs.metrics.FactorizedTopK(
        candidates=events.batch(128).map(event_model)
))

class RetrievalModel(tfrs.Model):

  def __init__(self):
    super().__init__()
    self.event_model: tf.keras.Model = event_model
    
    self.user_model: tf.keras.Model = user_model
    
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the event features and pass them into the event model,
    # getting embeddings back.
    positive_event_embeddings = self.event_model(features["event_id"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_event_embeddings)

## Ranking Model

In [67]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = user_model

    # Compute embeddings for events.
    self.event_embeddings = event_model

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    
  def call(self, inputs):

    user_id, event_id = inputs

    user_embedding = self.user_embeddings(user_id)
    event_embedding = self.event_embeddings(event_id)

    return self.ratings(tf.concat([user_embedding, event_embedding], axis=1))

task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

class EventModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = task

  # Call what model to use when making prediction
  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["event_id"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")
    
    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [68]:
# user id + event id = predicted rating
user_ids, event_ids = np.array([99 for i in range(1000)]), np.array([i for i in range(1,1001)])
predicted_list = list(np.concatenate(RankingModel()((user_ids, event_ids)).numpy()))
predicted_event_id = predicted_list.index(max(predicted_list))
print(f'predicted event id: {predicted_event_id}')
print(f'highest score: {max(predicted_list)}')

predicted event id: 201
highest score: 0.002127314917743206


# Model Fitting, Evaluating, Predicting

## Retrieval Model

### Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [69]:
retrieval_model = RetrievalModel()
retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

retrieval_model.fit(cached_train, epochs=1)

1/1 [==============================] - 0s 436ms/step - root_mean_squared_error: 0.0409 - loss: 0.0017 - regularization_loss: 0.0000e+00 - total_loss: 0.0017


In [70]:
retrieval_model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 117ms/step - root_mean_squared_error: 0.0412 - loss: 0.0017 - regularization_loss: 0.0000e+00 - total_loss: 0.0017


{'root_mean_squared_error': 0.041169263422489166,
 'loss': 0.0016949078999459743,
 'regularization_loss': 0,
 'total_loss': 0.0016949078999459743}

### Making predictions

Now that we have a retrieval_model, we would like to be able to make predictions. We can use the `tfrs.layers.factorized_top_k.BruteForce` layer to do this.

In [ ]:
user_id = 1
ratings_df.loc[ratings_df['user_id']==user_id]

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(retrieval_model.user_model, k=30)
# recommends events out of the entire events dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((events.batch(100), events.batch(100).map(retrieval_model.event_model)))
)

# Get k recommendations. INPUT MUST HAVE AT LEAST K ITEMS
i, titles = index(tf.constant([user_id]))
print(f"Recommendations for user {user_id}: {[int(i) for i in titles[0, :]]}")

In [ ]:
events_df.loc[events_df['id'].isin([i for i in titles[0,:].numpy()])]

## Ranking Model

Lowest: adam(lr=0.01)

Epoch 25/25

1/1 [==============================] - 0s 10ms/step - root_mean_squared_error: 0.6173 - loss: 0.3811 - regularization_loss: 0.0000e+00 - total_loss: 0.3811

In [ ]:
ranking_model = EventModel()
ranking_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

ranking_model.fit(cached_train, epochs=25)

In [ ]:
ranking_model.evaluate(cached_test, return_dict=True)

In [ ]:
import itertools

test_ratings = {}
test_event_ids = [i for i in range(1,event_count+1)]
for event_id in test_event_ids:
  test_ratings[event_id] = ranking_model({
      "user_id": np.array([39]),
      "event_id": np.array([event_id])
  })

print("Ratings Desc:")
for id, score in itertools.islice(sorted(test_ratings.items(), key=lambda x: x[1], reverse=True),10):
  title = events_df.loc[events_df['id']==id, 'event_name'].values[0]
  print(f"{title}: {score}")

print()

print("Ratings Asc:")
for id, score in itertools.islice(sorted(test_ratings.items(), key=lambda x: x[1]),10):
  title = events_df.loc[events_df['id']==id, 'event_name'].values[0]
  print(f"{title}: {score}")

# Model Serving

## Retrieval Model

After the model is trained, we need a way to deploy it.

In a two-tower retrieval model, serving has two components:

- a serving query model, taking in features of the query and transforming them into a query embedding, and
- a serving candidate model. This most often takes the form of an approximate nearest neighbours (ANN) index which allows fast approximate lookup of candidates in response to a query produced by the query model.

In [ ]:
# path = os.path.join(tmp, "model")
path = "/content/retrieval_model"

# Save the index.
tf.saved_model.save(index, path)

# Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load(path)

# Pass a user id in, get top predicted event titles back.
scores, titles = loaded([42])

print(f"Recommendations: {titles[0,:]}")

## Ranking Model

In [ ]:
import itertools

path = "/content/ranking_model"
tf.saved_model.save(ranking_model, path)
loaded = tf.saved_model.load(path)

test_ratings = {}
test_event_ids = [i for i in range(1,event_count+1)]
for event_id in test_event_ids:
    test_ratings[event_id] = loaded(
        {'user_id':tf.constant([42], dtype=tf.int64),
        'event_id':tf.constant([event_id], dtype=tf.int64)}
    )

print("Ratings Desc:")
for id, score in itertools.islice(sorted(test_ratings.items(), key=lambda x: list(x)[0], reverse=True),10):
  title = events_df.loc[events_df['id']==id, 'event_name'].values[0]
  print(f"{title}: {score}")

print()

print("Ratings Asc:")
for id, score in itertools.islice(sorted(test_ratings.items(), key=lambda x: list(x)[0]),10):
  title = events_df.loc[events_df['id']==id, 'event_name'].values[0]
  print(f"{title}: {score}")

In [ ]:
import shutil
folder='/content/retrieval_model'
shutil.make_archive(folder, 'zip', folder)